<a href="https://www.kaggle.com/code/jonbown/formula-1-post-race-summary?scriptVersionId=121401444" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<div align='center'>
    <img src="https://www.raconteur.net/wp-content/uploads/2018/04/shutterstock_762930772.jpg"/>
</div>

<style>
@font-face {
  font-family: F1;
  src: url(https://www.formula1.com/etc/designs/fom-website/fonts/F1Regular/Formula1-Regular.ttf);
}

span{
  font-family: F1;
}

a{
  font-family: F1;
}

.nav-pills>li.active>a:focus {
    color: #ffffff;
    background-color: lightgray;
}

.container-fluid, .container-fluid h1 {
    font-family: F1;
    line-height: 1.7;
}

.container-fluid p {
    font-family: F1;
    color:black;
}

h1,h2,h3,h4,h5,h6,p, table {
  font-family: F1;
  color: black
}
</style>

In [1]:
%%html
<style>
@font-face {
  font-family: F1;
  src: url(https://www.formula1.com/etc/designs/fom-website/fonts/F1Regular/Formula1-Regular.ttf);
}

span{
  font-family: F1;
}

a{
  font-family: F1;
}

.nav-pills>li.active>a:focus {
    color: #ffffff;
    background-color: lightgray;
}

.container-fluid, .container-fluid h1 {
    font-family: F1;
    line-height: 1.7;
}

.container-fluid p {
    font-family: F1;
    color:black;
}

h1,h2,h3,h4,h5,h6,p, table {
  font-family: F1;
  color: black
}
</style>

<div style="padding:10px; 
            color:##FF1801;
            margin:5px;
            font-size:150%;
            display:fill;
            border-radius:1px;
            border-style: solid;
            border-color:#FF1801;
            background-color:#2173B8;
            overflow:hidden;">
    <center>
        <a id='top'></a>
        <h3>Table of Contents</h3>
    </center>
    <br>
    <a href="#1"><h4 >1 -  Podium</h4></a>
    <a href="#2"><h4 >2 - Full Results</h4></a>
    <a href="#3"><h4 >3 - Qualifying</h4></a>
    <a href="#4"><h4 >4 - Pit Stops</h4></a>
    <a href="#5"><h4 >5 - Lap Summary</h4></a>
    <a href="#6"><h4>6 - Driver Standings</h4></a>
    <a href="#7"><h4>7 - Constructor Race Performance</h4></a>
    <a href="#8"><h4>8 - Constructor Standings</h4></a>
     <a href="#9"><h4>9 - Sources</h4></a>
</div>



In [2]:
!pip install pyergast

In [3]:
import numpy as np
import pandas as pd
import datetime
from datetime import date
import os
import seaborn as sns
from IPython.display import Markdown as md
import matplotlib.pyplot as plt
import plotly.express as px
from scipy import stats
import plotly.graph_objects as go
pd.set_option('display.precision', 1)
from pyergast import pyergast
import requests

In [4]:
def constructor_standings(year=None, race=None):
    """
    Fetch the constructor standings after a specific race in a specific year. Defaults to latest standings
    Parameters
    ----------
    year: int
        An optional parameter that specifies the year to be queried.
    race: int
        An optional parameter that specifies the round of a year to be queried.
    Returns
    -------
    pandas.DataFrame
    Index:
        RangeIndex
    Columns:
        position: int
        positionText: str
        points: int
        wins: int
        constructorID: str
        constructor: str
        nationality: str
    Example
    -------
    >>> pyergast.constructor_standings(1965)
       position positionText points wins    constructorID             name    nationality
    0         1            1     54    6     lotus-climax     Lotus-Climax        British
    1         2            2     45    3              brm              BRM        British
    2         3            3     27    0   brabham-climax   Brabham-Climax        British
    3         4            4     26    0          ferrari          Ferrari        Italian
    4         5            5     14    0    cooper-climax    Cooper-Climax        British
    5         6            6     11    1            honda            Honda       Japanese
    6         7            7      5    0      brabham-brm      Brabham-BRM        British
    7         8            8      2    0        lotus-brm        Lotus-BRM        British
    8         9            9      0    0     brabham-ford     Brabham-Ford        British
    9        10           10      0    0             alfa       Alfa Romeo        Italian
    10       11           11      0    0   lds-alfa_romeo   LDS-Alfa Romeo  South African
    11       12           12      0    0      cooper-ford      Cooper-Ford        British
    12       13           13      0    0       lds-climax       LDS-Climax  South African
    13       14           14      0    0       lotus-ford       Lotus-Ford        British
    14       15           15      0    0               re               RE      Rhodesian
    15       16           16      0    0  cooper-maserati  Cooper-Maserati        British
    """
    if year and race:
        assert year >= 1958, 'Constructor standings only available starting 1958'
        url = 'http://ergast.com/api/f1/{}/{}/constructorStandings.json?limit=1000'.format(year, race)
    elif year:
        assert year >= 1958, 'Constructor standings only available starting 1958'
        url = 'http://ergast.com/api/f1/{}/constructorStandings.json?limit=1000'.format(year, race)
    else:
        url = 'http://ergast.com/api/f1/current/constructorStandings.json?limit=1000'

    r = requests.get(url)
    assert r.status_code == 200, 'Cannot connect to Ergast API. Check your inputs.'
    constructorStandings = r.json()['MRData']['StandingsTable']['StandingsLists'][0]['ConstructorStandings']

    for constructor in constructorStandings:
        constructor['constructorID'] = constructor['Constructor']['constructorId']
        constructor['name'] = constructor['Constructor']['name']
        constructor['nationality'] = constructor['Constructor']['nationality']
        del constructor['Constructor']

    return pd.DataFrame(constructorStandings)



def get_pit_stops(year=None, race=None):
    """
    """
    if year and race:
        url = 'http://ergast.com/api/f1/{}/{}/pitstops.json?limit=1000'.format(year, race)
    r = requests.get(url)

    assert r.status_code == 200, 'Cannot connect to Ergast API'
    drivers = r.json()
    result = pd.DataFrame(drivers["MRData"]["RaceTable"]["Races"][0]["PitStops"])

    return result


def get_round(year=None, race=None):
    """
    """
    if year and race:
        url = 'http://ergast.com/api/f1/{}/{}/pitstops.json?limit=1000'.format(year, race)
    r = requests.get(url)

    assert r.status_code == 200, 'Cannot connect to Ergast API'
    drivers = r.json()
    result = pd.DataFrame(drivers["MRData"]["RaceTable"]["Races"][0]["PitStops"])

    return result

def get_lap_times(year=None, race=None):
    """
    """
    if year and race:
        url = 'http://ergast.com/api/f1/{}/{}/laps.json?limit=1000'.format(year, race)
    r = requests.get(url)

    assert r.status_code == 200, 'Cannot connect to Ergast API'
    drivers = r.json()
    result = pd.DataFrame(drivers["MRData"]["RaceTable"]["Races"][0]["Laps"])
    lap_df = pd.DataFrame()
    for lap in result.number.unique():
        temp_df = pd.DataFrame(result[result['number'] == lap].Timings.iloc[0])
        temp_df['lap'] = lap
        lap_df = pd.concat([lap_df, temp_df], ignore_index=True, axis=0)

    return lap_df

def get_last_round():
    """
    """
    url = 'http://ergast.com/api/f1/current/last/results.json?limit=1000'
    r = requests.get(url)

    assert r.status_code == 200, 'Cannot connect to Ergast API'
    drivers = r.json()
    result = drivers["MRData"]["RaceTable"]["round"]
   

    return int(result)

In [5]:
#DATA CLEANING
my_date = date.today()
year, week_num, day_of_week = my_date.isocalendar()
race = get_last_round()
#races = pd.read_csv('/kaggle/input/formula-1-world-championship-1950-2020/races.csv')
race_result = pyergast.get_race_result()
race_result['points'] = race_result.points.astype(float)
#circuits = pd.read_csv('/kaggle/input/formula-1-world-championship-1950-2020/circuits.csv')
circuits = pyergast.get_circuits()
#constructor_standings = pd.read_csv('/kaggle/input/formula-1-world-championship-1950-2020/constructor_standings.csv')
constructor_overall = constructor_standings(year=year)
constructor_overall['points'] = constructor_overall.points.astype(float)
constructor_results = constructor_standings(year=2022, race=race)
constructor_results['points'] = constructor_results.points.astype(float)
#DRIVERS
drivers = pyergast.get_drivers(year=year)
drivers_2 = pd.read_csv('/kaggle/input/formula-1-world-championship-1950-2020/drivers.csv')
drivers_2 ['Driver'] = drivers_2['code']
drivers_2['driverID'] = drivers_2['driverRef']
drivers['driverID'] = drivers['driverId']
drivers_full = drivers.merge(drivers_2, on='driverID', how='left', suffixes=('_1', '_2'))
#constructors = pd.read_csv('/kaggle/input/formula-1-world-championship-1950-2020/constructors.csv')
constructors = pyergast.get_constructors()
lap_times = get_lap_times(year, race)
lap_times['driverID'] = lap_times['driverId']
#status = pd.read_csv('/kaggle/input/formula-1-world-championship-1950-2020/status.csv')
#driver_standings = pd.read_csv('/kaggle/input/formula-1-world-championship-1950-2020/driver_standings.csv')
driver_standings = pyergast.driver_standings()
pit_stops = get_pit_stops(year, race)
pit_stops['driverID'] = pit_stops['driverId']
#sprint_results = pd.read_csv('/kaggle/input/formula-1-world-championship-1950-2020/sprint_results.csv')
#results = pd.read_csv('/kaggle/input/formula-1-world-championship-1950-2020/results.csv')
#qualifying = pd.read_csv('/kaggle/input/formula-1-world-championship-1950-2020/qualifying.csv')
qualifying = pyergast.get_qualifying_result()
schedule = pyergast.get_schedule()


In [6]:
#Helpers
def get_sec(time_str):
    """Get seconds from time."""
    if (time_str != '\\N' or not time_str):
        m, s = str(time_str).split(':')
        return float(m) * 60 + float(s)

#Colors
color_map = {
        'ALB': '#00A0DE',
        'SAR': '#00A0DE',
        'ZHO': '#A42134',
        'BOT': '#A42134',
        'TSU': '#20394C',
        'DEV': '#20394C',
        'VER': '#000B8D',
        'PER': '#000B8D',
        'HAM': '#00A19B',
        'RUS': '#00A19B',
        'MAG': '#E6002B',
        'HUL': '#E6002B', 
          'ALO': '#002420', 
          'STR': '#002420', 
          'LEC': '#EF1A2D',
          'SAI':'#EF1A2D',
          'OCO': '#2173B8',
          'GAS': '#2173B8',
        'NOR': '#FF8000' ,
        'PIA': '#FF8000'
}

In [7]:
# DATA CLEANING
lap_times['duration'] = lap_times['time'].apply(get_sec)

current_circuit = schedule[schedule['round'] == str(race)].circuitID.values[0]
current_race_name =schedule[schedule['round'] == str(race)].circuitName.values[0]


#DRIVER/LAP
lap_driver = lap_times.merge(drivers_full, on='driverID', how='left', suffixes=('_1', '_2'))
lap_driver['lap'] = lap_driver.lap.astype(float)
lap_driver_no = lap_driver[(stats.zscore(lap_driver['duration'])) <1]


#RESULTS
num_laps = max(race_result['laps'])
merged = race_result.merge(drivers_full, on='driverID', how='left', suffixes=('_1', '_2'))
res_race_laps = merged.copy()
res_race_laps['laps'] = 0
reduced = res_race_laps[['laps', 'grid', 'Driver']]
reduced.columns = ['laps', 'position', 'Driver']
reduced_res = pd.concat([reduced, merged[['laps', 'position', 'Driver']]], axis=0)



track = circuits[circuits['circuitId'] == current_circuit]
track_lat = track['Latitude'].values[0]
track_lng = track['Longtitude'].values[0]


#PODIIUM
res_race_lap_status_pod = race_result[race_result['position'] != '\\N'].copy()
res_race_lap_status_pod = res_race_lap_status_pod.merge(drivers_full, on='driverID', how='left', suffixes=('_1', '_2'))
res_race_lap_status_pod['position'] = res_race_lap_status_pod['position'].astype(int)
res_race_lap_status_pod = res_race_lap_status_pod.sort_values(by='position')


#PIT STOPS
pit_stops['duration'] = pit_stops['duration'].astype(float)
driver_group = pd.DataFrame()
pit_stops = pit_stops.merge(drivers_full, on='driverID', how='left', suffixes=('_1', '_2'))
mean = pit_stops.groupby(['Driver'])['duration'].mean()
mini = pit_stops.groupby(['Driver'])['duration'].min()
maxi = pit_stops.groupby(['Driver'])['duration'].max()
driver_group['mean'] = mean
driver_group['max'] = maxi
driver_group['min'] = mini
driver_group = driver_group.sort_values(by="max")
driver_group['Driver'] = maxi.index

# QUALIFYING & FINISHING
qualifying = qualifying.fillna('\\N')
qualifying['q1_dur'] = qualifying['Q1'].apply(get_sec)
qualifying['q2_dur'] = qualifying['Q2'].apply(get_sec)
qualifying['q3_dur'] = qualifying['Q3'].apply(get_sec)
qualifying['max_time'] = qualifying.apply(lambda x: max(x['q1_dur'], x['q2_dur'], x['q3_dur']), axis=1)
qualifying['min_time'] = qualifying.apply(lambda x: min(x['q1_dur'], x['q2_dur'], x['q3_dur']), axis=1)
max_time = max(qualifying['max_time']) + 1
min_time = min(qualifying['min_time']) - 1
qualifying = qualifying.merge(drivers_full, on='driverID', how='left', suffixes=('_1', '_2'))

In [8]:
display(md(f"# {current_race_name}, {year} Data Summary"))

# Bahrain International Circuit, 2023 Data Summary

In [9]:
!pip install gmplot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.7/164.7 kB 5.3 MB/s eta 0:00:00


In [10]:
fig = px.scatter_mapbox(track, lat="Latitude", lon="Longtitude",
                        color_discrete_sequence=["fuchsia"], zoom=12, height=300)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

<a id="1"></a>
<h1 style='background:#FF1801;border:0; color:black;
    box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75);
    transform: rotateX(10deg);
    '><center style='color: #3E3D53;'>Podium</center></h1>

In [11]:
# Styling
def podium(styler):
    styler.set_caption("Podium Results")
    styler.background_gradient(axis=None, vmin=1, vmax=5, cmap="YlGnBu")
    styler.hide_index()
   # styler.format({'points': "{.1f}"})
    return styler


def results_style(styler):
    styler.set_caption("Race Results")
    styler.background_gradient(axis=None, vmin=1, vmax=5, cmap="YlGnBu")
    styler.hide_index()
    return styler

def driver_stand(styler):
    styler.set_caption("Driver Standings")
    styler.background_gradient(axis=None, vmin=1, vmax=5, cmap="YlGnBu")
    styler.hide_index()
    return styler

def const_standings(styler):
    styler.set_caption("Constructor Standings")
    styler.background_gradient(axis=None, vmin=1, vmax=5, cmap="YlOrRd")
    styler.hide_index()
    return styler

In [12]:
res_race_lap_status_pod[['Driver', 'grid', 'position', 'points']].iloc[:3].style.pipe(podium)

Driver,grid,position,points
VER,1,1,25.0
PER,2,2,18.0
ALO,5,3,15.0


<a id="2"></a>
<h1 style='background:#FF1801;border:0; color:black;
    box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75);
    transform: rotateX(10deg);
    '><center style='color: #3E3D53;'>Full Results</center></h1>

In [13]:
res_race_lap_status_pod[['Driver', 'grid', 'position', 'points', 'status']].style.pipe(results_style)

Driver,grid,position,points,status
VER,1,1,25.0,Finished
PER,2,2,18.0,Finished
ALO,5,3,15.0,Finished
SAI,4,4,12.0,Finished
HAM,7,5,10.0,Finished
STR,8,6,8.0,Finished
RUS,6,7,6.0,Finished
BOT,12,8,4.0,Finished
GAS,20,9,2.0,Finished
ALB,15,10,1.0,Finished


<a id="3"></a>
<h1 style='background:#FF1801;border:0; color:black;
    box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75);
    transform: rotateX(10deg);
    '><center style='color: #3E3D53;'>Qualifying</center></h1>

In [14]:
fig = go.Figure(layout_yaxis_range=[min_time, max_time])
fig.add_trace(go.Bar(
    x=qualifying['Driver'],
    y=qualifying['q1_dur'],
    name='Q1',
    marker_color='red'
))
fig.add_trace(go.Bar(
    x=qualifying['Driver'],
    y=qualifying['q2_dur'],
    name='Q2',
    marker_color='gold'
))
fig.add_trace(go.Bar(
    x=qualifying['Driver'],
    y=qualifying['q3_dur'],
    name='Q3',
    marker_color='orange'
))

fig.update_layout(barmode='group', xaxis_tickangle=-45, title="Qualifying Results")
fig.show()

In [15]:
#start/finish path
lap_driver_sf = reduced_res[(reduced_res['laps'] == 0) | (reduced_res['laps'] == num_laps)].copy()
lap_driver_sf['position'] = lap_driver_sf['position'].astype(float)

df = px.data.gapminder().query("continent=='Oceania'")
fig = px.line(lap_driver_sf, x="laps", y="position", color='Driver', color_discrete_map=color_map, title="Qualifying to Finish", labels={
    "position": "Driver",
    "laps":"Lap (Start/Finish Only)"
})
fig.update_layout(title_text='<b>Qualifying To Finish</b>', titlefont = dict(
         family = 'Arial, sans-serif',
         size = 30
      ), title_x=0.5)
fig.show()

<a id="4"></a>
<h1 style='background:#FF1801;border:0; color:black;
    box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75);
    transform: rotateX(10deg);
    '><center style='color: #3E3D53;'>Pit Stops</center></h1>

In [16]:
max_pit = max(driver_group['max']) + 1
min_pit = min(driver_group['min']) - 1

data = [
go.Bar(y=driver_group['max'], x=driver_group.Driver, name='Max Pit',  
            #base=0
           ),
go.Bar(y=driver_group['mean'], x=driver_group.Driver, name='Avg PIt'),
go.Bar(y=driver_group['min'], x=driver_group.Driver, name='Min PIt')]

layout = go.Layout(
    barmode='overlay',
    title="Pit Times",
    title_text='<b>Pit Stop Times</b>', titlefont = dict(
         family = 'Arial, sans-serif',
         size = 30
      ), title_x=0.5
)

fig = dict(data = data, layout = layout, layout_yaxis_range=[min_pit, max_pit])
from plotly.offline import init_notebook_mode, iplot
iplot(fig, show_link=False)

<a id="5"></a>
<h1 style='background:#FF1801;border:0; color:black;
    box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75);
    transform: rotateX(10deg);
    '><center style='color: #3E3D53;'>Lap Results</center></h1>

In [17]:
fig = px.box(lap_driver_no, x="duration", y="Driver", color='Driver', points=False, color_discrete_map=color_map, labels={
    "code": "Driver",
    "duration":"Duration (Seconds)"
})
fig.update_layout(showlegend=False)
fig.update_layout(title_text='<b>Lap Times</b>', titlefont = dict(
         family = 'Arial, sans-serif',
         size = 30
      ), title_x=0.5)
fig.show()

In [18]:
def multi_lap_plot(names, addAll = True):
    fig = go.Figure()

    fig = px.line(lap_driver_no, x="lap", y="duration", color='Driver', labels={
    "duration": "Lap Time (In seconds, Outliers Removed)",
    "lap": "Lap"
}, title='Lap Times', color_discrete_map=color_map)
    fig.update_layout(title_text='<b>Lap Times</b>', titlefont = dict(
         family = 'Arial, sans-serif',
         size = 30
      ), title_x=0.5)

    button_all = dict(label = 'All',
                      method = 'update',
                      args = [{'visible': [True]*len(names),
                               'title': 'All',
                               'showlegend':True}])

    def create_layout_button(column):
        return dict(label = column,
                    method = 'update',
                    args = [{'visible': [c == column for c in names],
                             'title': column,
                             'showlegend': True}])

    fig.update_layout(
        updatemenus=[go.layout.Updatemenu(
            active = 0,
            buttons = ([button_all] * addAll) + [create_layout_button(column) for column in names]
            )
        ])
    
    fig.show()
    
    
    
def multi_pos_plot(names, addAll = True):
    fig = go.Figure()

    fig = px.line(lap_driver, x="lap", y="position", color='Driver', labels={
    "position":  "Position",
    "lap": "Lap"
}, color_discrete_map=color_map)
    fig.update_layout(title_text='<b>Position By Lap</b>', titlefont = dict(
         family = 'Arial, sans-serif',
         size = 30
      ), title_x=0.5)

    button_all = dict(label = 'All',
                      method = 'update',
                      args = [{'visible': [True]*len(names),
                               'title': 'All',
                               'showlegend':True}])

    def create_layout_button(column):
        return dict(label = column,
                    method = 'update',
                    args = [{'visible': [c == column for c in names],
                             'title': column,
                             'showlegend': True}])

    fig.update_layout(
        updatemenus=[go.layout.Updatemenu(
            active = 0,
            buttons = ([button_all] * addAll) + [create_layout_button(column) for column in names]
            )
        ])
    
    fig.show()

In [19]:
mapping = list(color_map.keys())
mapping.sort()
multi_lap_plot(mapping)

In [20]:
multi_pos_plot(mapping)

<a id="6"></a>
<h1 style='background:#FF1801;border:0; color:black;
    box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75);
    transform: rotateX(10deg);
    '><center style='color: #3E3D53;'>Driver Standings</center></h1>

In [21]:
drive_stand = driver_standings.merge(drivers_full, on='driverID', how='left', suffixes=('_1', '_2'))
drive_stand['points'] = drive_stand.points.astype(float)
drive_stand_race  = drive_stand .sort_values(by='points', ascending=False)
drive_stand_race[['Driver', 'points']].style.pipe(driver_stand)

Driver,points
VER,25.0
PER,18.0
ALO,15.0
SAI,12.0
HAM,10.0
STR,8.0
RUS,6.0
BOT,4.0
GAS,2.0
ALB,1.0


<a id="7"></a>
<h1 style='background:#FF1801;border:0; color:black;
    box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75);
    transform: rotateX(10deg);
    '><center style='color: #3E3D53;'>Constructors Post-Race</center></h1>

In [22]:
const_stand_race = constructor_results.sort_values(by='points', ascending=False)
const_stand_race[['name', 'points']].style.pipe(const_standings)

name,points
Ferrari,44.0
Mercedes,27.0
Haas F1 Team,10.0
Alfa Romeo,9.0
Alpine F1 Team,8.0
AlphaTauri,4.0
Aston Martin,0.0
Williams,0.0
McLaren,0.0
Red Bull,0.0


<a id="8"></a>
<h1 style='background:#FF1801;border:0; color:black;
    box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75);
    transform: rotateX(10deg);
    '><center style='color: #3E3D53;'>Constructor Standings</center></h1>

In [23]:
const_stand_race = constructor_overall.sort_values(by='points', ascending=False)
const_stand_race[['name', 'points']].style.pipe(const_standings)

name,points
Red Bull,43.0
Aston Martin,23.0
Mercedes,16.0
Ferrari,12.0
Alfa Romeo,4.0
Alpine F1 Team,2.0
Williams,1.0
AlphaTauri,0.0
Haas F1 Team,0.0
McLaren,0.0


<h1 style='background:#FF1801;border:0; color:black;
    box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75);
    transform: rotateX(10deg);
    '><center style='color: #3E3D53;'>Thank You</center></h1>

### Please upvote if you liked this notebook and comment with your feedback! Thank you so much for supporting my work. 

<a id="9"></a>
<h1 style='background:#FF1801;border:0; color:black;
    box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75);
    transform: rotateX(10deg);
    '><center style='color: #3E3D53;'>Acknowledgements</center></h1>

- Shoutout to @VOPANI for creating this fantastic, F1 dataset
- The F1 Developer API http://ergast.com/mrd/
- The python package ergast API wrapper, [[pyergast](https://github.com/weiranyu/pyErgast)]
- Very inspired by this [[notebook](https://www.kaggle.com/code/ekrembayar/formula-1-70th-anniversary)] on the same dataset

<div class="row" style='float:left' align='center'>
<div class="column" align = 'center' style='float:left; max-width:50%;'> 
    <img src="https://cdn-1.motorsport.com/images/mgl/Y99JQRbY/s800/red-bull-racing-logo-1.jpg" width=70>
    <img src="https://www.formula1.com/content/fom-website/en/teams/Alpine/_jcr_content/logo.img.jpg/1673451453749.jpg" width=70>
    <img src="https://cdn-8.motorsport.com/images/mgl/YEQJgyLY/s800/mclaren-f1-team-logo-1.jpg" width=60>
    <img src="https://assets.turbologo.com/blog/en/2019/10/19084952/ferrari-logo-illustration-958x575.jpg" width=70 />
      <img src="https://cdn.sanity.io/images/fnx611yr/production/ff454bb2b4adb541b66477adcb49088f8de0fb4a-2400x2400.jpg?rect=0,572,2400,1256&w=1200&h=628&auto=format"/ width=70>
    <img src="https://www.formula1.com/content/fom-website/en/teams/Mercedes/_jcr_content/logo.img.jpg/1582638425211.jpg" width=70/>
    <img src="https://diginomica.com/sites/default/files/images/2019-05/Haas-F1-logo.jpg" width=70>
    <img src="https://d3cm515ijfiu6w.cloudfront.net/wp-content/uploads/2021/01/01082241/Aston-Martin.jpg" width=70>
    <img src="https://www.formula1.com/content/fom-website/en/teams/AlphaTauri/_jcr_content/logo.img.jpg/1582650557134.jpg" width=70>
      <img src="https://www.formula1.com/content/dam/fom-website/manual/teams/Sauber/Alfa_Romeo_Racing_logo.jpg"/ width=70>
</div>
 <div class="column" style='float:left; max-width:50%;'> 
    <img src="https://corp.formula1.com/wp-content/uploads/2022/09/Capture.jpg" />
</div>
 </div>